In [34]:
import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim
import sys
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
from datetime import datetime
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

In [2]:
import platform; print(platform.platform())

macOS-14.6-arm64-arm-64bit


In [3]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    # print(X.shape,T,N,Y.shape)
    df = np.array(X)
    print(df.shape)
    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY

def torch_data(x, y):
    x = torch.from_numpy(x)
    x = torch.unsqueeze(x, 1)
    y = torch.from_numpy(y)
    y = F.one_hot(y, num_classes=3)
    return x, y

In [4]:
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, k, num_classes, T):
        """Initialization""" 
        self.k = k
        self.num_classes = num_classes
        self.T = T
            
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [5]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [6]:
device = mps_device

In [7]:
x.type()

'torch.mps.FloatTensor'

In [8]:
# please change the data_path to your local path
# data_path = '/nfs/home/zihaoz/limit_order_book/data'

dec_data = np.loadtxt('/Users/jandh/Downloads/data (1)/Train_Dst_NoAuction_DecPre_CF_7.txt')
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

dec_test1 = np.loadtxt('/Users/jandh/Downloads/data (1)/Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test2 = np.loadtxt('/Users/jandh/Downloads/data (1)/Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('/Users/jandh/Downloads/data (1)/Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

print(dec_train.shape, dec_val.shape, dec_test.shape)

(149, 203800) (149, 50950) (149, 156516)


In [9]:
sys.getsizeof(dec_data)/1000000

303.662128

In [10]:
batch_size = 64

dataset_train = Dataset(data=dec_train, k=4, num_classes=3, T=100)
dataset_val = Dataset(data=dec_val, k=4, num_classes=3, T=100)
dataset_test = Dataset(data=dec_test, k=4, num_classes=3, T=100)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

(203800, 40)
(50950, 40)
(156516, 40)
torch.Size([203701, 1, 100, 40]) torch.Size([203701])


In [11]:
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

tensor([[[[0.1276, 0.0100, 0.1274,  ..., 0.0150, 0.1265, 0.0332],
          [0.1276, 0.0100, 0.1274,  ..., 0.0150, 0.1265, 0.0332],
          [0.1276, 0.0100, 0.1274,  ..., 0.0150, 0.1265, 0.0332],
          ...,
          [0.1272, 0.0052, 0.1270,  ..., 0.0146, 0.1261, 0.0464],
          [0.1272, 0.0052, 0.1270,  ..., 0.0146, 0.1261, 0.0464],
          [0.1272, 0.0052, 0.1271,  ..., 0.0146, 0.1262, 0.0025]]]],
       dtype=torch.float64)
tensor([0.], dtype=torch.float64)
torch.Size([1, 1, 100, 40]) torch.Size([1])


In [12]:
# #CPU version
# class deeplob(nn.Module):
#     def __init__(self, y_len):
#         super().__init__()
#         self.y_len = y_len
        
#         # convolution blocks
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
#             nn.LeakyReLU(negative_slope=0.01),
# #             nn.Tanh(),
#             nn.BatchNorm2d(32),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(32),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(32),
#         )
#         self.conv2 = nn.Sequential(
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
#             nn.Tanh(),
#             nn.BatchNorm2d(32),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
#             nn.Tanh(),
#             nn.BatchNorm2d(32),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
#             nn.Tanh(),
#             nn.BatchNorm2d(32),
#         )
#         self.conv3 = nn.Sequential(
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,10)),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(32),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(32),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(32),
#         )
        
#         # inception moduels
#         self.inp1 = nn.Sequential(
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(64),
#             nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(64),
#         )
#         self.inp2 = nn.Sequential(
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(64),
#             nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(64),
#         )
#         self.inp3 = nn.Sequential(
#             nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.BatchNorm2d(64),
#         )
        
#         # lstm layers
#         self.lstm = nn.LSTM(input_size=192, hidden_size=64, num_layers=1, batch_first=True)
#         self.fc1 = nn.Linear(64, self.y_len)

#     def forward(self, x):
#         # h0: (number of hidden layers, batch size, hidden size)
#         h0 = torch.zeros(1, x.size(0), 64).to(device)
#         c0 = torch.zeros(1, x.size(0), 64).to(device)
    
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
        
#         x_inp1 = self.inp1(x)
#         x_inp2 = self.inp2(x)
#         x_inp3 = self.inp3(x)  
        
#         x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)
        
# #         x = torch.transpose(x, 1, 2)
#         x = x.permute(0, 2, 1, 3)
#         x = torch.reshape(x, (-1, x.shape[1], x.shape[2]))
        
#         x, _ = self.lstm(x, (h0, c0))
#         x = x[:, -1, :]
#         x = self.fc1(x)
#         forecast_y = torch.softmax(x, dim=1)
        
#         return forecast_y

In [13]:
class deeplob(nn.Module):
    def __init__(self, y_len):
        super().__init__()
        self.y_len = y_len
        
        # convolution blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
#             nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,10)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        
        # inception moduels
        self.inp1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        
        # lstm layers
        self.lstm = nn.LSTM(input_size=192, hidden_size=64, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(64, self.y_len)

    def forward(self, x):
        # h0: (number of hidden layers, batch size, hidden size)
        h0 = torch.zeros(1, x.size(0), 64).to(device)
        c0 = torch.zeros(1, x.size(0), 64).to(device)
    
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x_inp1 = self.inp1(x)
        x_inp2 = self.inp2(x)
        x_inp3 = self.inp3(x)  
        
        x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)
        
#         x = torch.transpose(x, 1, 2)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = torch.reshape(x, (-1, x.shape[1], x.shape[2]))
        
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc1(x)
        forecast_y = torch.softmax(x, dim=1)
        
        return forecast_y

In [14]:
# # testing gradiant flows for CPU and GPU

# model = deeplob(y_len = dataset_train.num_classes)
# model2 = deeplob(y_len = dataset_train.num_classes)
# model2.load_state_dict(model.state_dict())
# # print(torch.allclose(model.weight, model2.weight))

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.0001)

# data_iter = iter(train_loader)
# inputs , targets = next(data_iter)
# device = 'cpu'
# # print(inputs)
#  # = train_loader.next()
# optimizer.zero_grad()
# outputs = model(inputs.to(torch.float))
# loss = criterion(outputs, targets.to(dtype=torch.int64))
# print(loss)
# loss.backward()
# optimizer.step()

# print(model.inp3[1].weight.grad.max())

# for name, param in model.lstm.named_parameters():
#     if 'weight' in name:
#         print(name, param.grad.max()) 

# for name, param in model2.lstm.named_parameters():
#     if 'weight' in name:
#         print(name, param.grad.max()) 

# print(model2.inp3[1].weight.grad.max())

# device = mps_device
# model2.to(mps_device)
# # inputs , targets = next(data_iter)
# optimizer2.zero_grad()
# outputs2 = model2(inputs.to(mps_device,dtype = torch.float32))
# loss2 = criterion(outputs2, targets.to(mps_device,dtype=torch.int64))
# print(loss2)
# loss2.backward()
# optimizer2.step()

# def compare_models(model_1, model_2):
#     models_differ = 0
#     for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
#         if torch.equal(key_item_1[1], key_item_2[1]):
#             pass
#         else:
#             models_differ += 1
#             if (key_item_1[0] == key_item_2[0]):
#                 print('Mismtach found at', key_item_1[0])
#             else:
#                 raise Exception
#     if models_differ == 0:
#         print('Models match perfectly! :)')

In [15]:
model = deeplob(y_len = dataset_train.num_classes)
model.to(mps_device)
device = mps_device

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
state_dict = None

In [17]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    global state_dict
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0

    for it in tqdm(range(epochs)):
        
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # move data to GPU
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
            # print("inputs.shape:", inputs.shape)
            # zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            # print("about to get model output")
            outputs = model(inputs)
            # print("done getting model output")
            # print("outputs.shape:", outputs.shape, "targets.shape:", targets.shape)
            loss = criterion(outputs, targets)
            # Backward and optimize
            # print("about to optimize")
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        # Get train loss and test loss
        train_loss = np.mean(train_loss) # a little misleading
    
        model.eval()
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        if test_loss < best_test_loss:
            # state_dict = copy.deepcopy(model.state_dict())
            torch.save(model, '/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/best_val_model_pytorch')
            best_test_loss = test_loss
            best_test_epoch = it
            print('model saved')

        dt = datetime.now() - t0
        print(dt)
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')

    return train_losses, test_losses

In [18]:
# import multiprocessing
# multiprocessing.set_start_method("fork")

In [19]:
train_losses, val_losses = batch_gd(model, criterion, optimizer, 
                                    train_loader, val_loader, epochs=50)

  2%|▊                                        | 1/50 [02:59<2:26:29, 179.37s/it]

model saved
0:02:59.369835
Epoch 1/50, Train Loss: 0.9309,           Validation Loss: 1.0139, Duration: 0:02:59.369835, Best Val Epoch: 0


  4%|█▌                                     | 2/50 [36:13<16:37:24, 1246.76s/it]

model saved
0:33:13.927446
Epoch 2/50, Train Loss: 0.8378,           Validation Loss: 0.9866, Duration: 0:33:13.927446, Best Val Epoch: 1


  6%|██▎                                    | 3/50 [53:57<15:11:26, 1163.53s/it]

...

0:03:12.560456
Epoch 48/50, Train Loss: 0.6529,           Validation Loss: 0.9129, Duration: 0:03:12.560456, Best Val Epoch: 17


 98%|███████████████████████████████████████▏| 49/50 [3:27:02<03:16, 196.45s/it]

0:03:04.530532
Epoch 49/50, Train Loss: 0.6522,           Validation Loss: 0.9066, Duration: 0:03:04.530532, Best Val Epoch: 17


100%|████████████████████████████████████████| 50/50 [3:30:09<00:00, 252.18s/it]

0:03:06.364955
Epoch 50/50, Train Loss: 0.6516,           Validation Loss: 0.9113, Duration: 0:03:06.364955, Best Val Epoch: 17


In [20]:
print(train_losses)

[0.93093095 0.83776069 0.80920504 0.78910695 0.77382254 0.76214061
 0.75219157 0.74287189 0.73526348 0.72854619 0.72283292 0.71770586
 0.71298178 0.70938897 0.70495707 0.70183109 0.69865748 0.69589203
 0.69356334 0.69114647 0.68799215 0.68588215 0.68341167 0.68117793
 0.67981741 0.67856626 0.67657444 0.67510226 0.67278343 0.67135176
 0.67060535 0.66925408 0.66816197 0.66660362 0.66493018 0.66456361
 0.66254729 0.66221397 0.66063586 0.65972054 0.65842114 0.65782859
 0.65711623 0.65625063 0.65567259 0.65481501 0.65385798 0.65290727
 0.65219235 0.65159853]


In [30]:
# import pickle
# with open('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/train_losses.pkl', 'wb') as f:
#     pickle.dump(train_losses, f)
# with open('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/val_losses.pkl', 'wb') as f:
#     pickle.dump(val_losses, f)

In [33]:
plt.figure(figsize=(15,6))
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.legend()

## Model_Testing

In [23]:
model = torch.load('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/best_val_model_pytorch')

n_correct = 0.
n_total = 0.
for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_2491/1323947167.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/Users/jandh/Desktop/Old D

Test acc: 0.7334


In [24]:
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)   

In [27]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.7334369026384601
              precision    recall  f1-score   support

           0     0.6764    0.7627    0.7170     51924
           1     0.8279    0.7104    0.7647     53532
           2     0.7142    0.7278    0.7209     50961

    accuracy                         0.7334    156417
   macro avg     0.7395    0.7336    0.7342    156417
weighted avg     0.7406    0.7334    0.7346    156417

